# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import seaborn as sns

import warnings
warnings.simplefilter("ignore", UserWarning)

## Baca data
File `*.csv` dan `*.ipynb` berada dalam folder yang sama untuk memudahkan pembacaan data

In [ ]:
# Read data
data = pd.read_csv('data_megaentry_2023.csv')

## Mengganti format dan nama
Menjadikan format tanggal dengan menggunakan `pd.to_datetime`.

Mengganti nama dengan menggunakan `rename`.

In [ ]:
# Mengganti format DATA TIMESTAMP menjadi format tanggal
data['DATE'] = pd.to_datetime(data['DATA TIMESTAMP'])

# Mengganti nama header sesuai keinginan
data = data.rename(columns={'RAINFALL 24H RRRR': 'RRRR', 'TEMP MIN C TNTNTN': 'TNTN'})

## Memilih data dan waktu
Memilih data dengan syarat jam bisa menggunakan `dt.hour`. Jika ingin memilih data dengan syarat bulan bisa menggunakna `dt.month`. 

Hal ini bisa di lakukan karena kita sudah mengubah format DATE menjadi format tanggal.


In [ ]:
# Memilih data tanggal, hujan dan suhu minimum
data_24 = data[['DATE', 'RRRR', 'TNTN']]

# Memilih data pada pukul 00 UTC
data_24 = data_24[data_24['DATE'].dt.hour == 0]

## Filter data curah hujan
Filter data curah hujan di atas 0 mm.

In [ ]:
# Memfilter data hujan di atas 0 mm. Supaya data hujan 0 itu diabaikan atau hilang.
data_24_non_0 = data_24[data_24['RRRR'] > 0]

Ada beberapa cara Filter waktu yang dapat digunakan, seperti Filter data curah hujan berdasarkan bulan dapat menggunakan `dt.month`. Filter dengan menggunakan urutan waktu bisa menggunakan `iloc`. Filter data curah hujan berdasarkan spesific tanggal `dt.date`. 



In [ ]:
# Filter berdasarkan bulan
feb_data = data_24[data_24['DATE'].dt.month == 2]

# Filter berdasarkan urutan waktu
sel_data = data_24.iloc[1:91]

# Filter berdasarkan waktu spesifik
start_date = pd.to_datetime('2023-02-01T00:00:00.000Z')
end_date = pd.to_datetime('2023-06-01T00:00:00.000Z')

# Filter untuk 1 tanggal
select_1_date = data_24[data_24['DATE'].dt.date == start_date.date()]

# Filter untuk range data antara tanggal mulai dan akhir
range_date = data_24[(data_24['DATE'] >= start_date) & (data_24['DATE'] <= end_date)]

# Filter untuk beberapa tanggal
multi_date = data_24[data_24['DATE'].dt.date.isin([start_date.date(), end_date.date()])]

## Replace data
mereplace suatu nilai dapat menggunakan `replace`. Jika tidak ada nilai *(NaN)*, dapat kita ubah menjadi sebuah nilai dengan menggunakan `fillna`. Jika ingin menghapus semua data *NaN*, bisa menggunakan `dropna`.  

In [ ]:
# Replace nilai RRRR 9999 dan 8888 menjadi 0
data_24['RRRR'] = data_24['RRRR'].replace([9999, 8888], 0)

# Menghapus NaN data agar tidak dihitung
data_24_non_nan = data_24.dropna()

# Replace NaN data menjadi 0
data_24 = data_24.fillna(0)

# mengganti nama agar mempermudah
time = data_24['DATE']
prec = data_24['RRRR']
tmin = data_24['TNTN']

## Pengolahan Statistik
Ada berbagai pengolahan statistik sederhana yang dapat digunakan seperti `mean, max, min, variance, standard deviation, skewness, kurtosis, sum` dan `percentile`.

In [ ]:
# data length
count_data = len(prec)
print(f'Length of data: {count_data}')

# Mean data
data_mean = np.mean(prec)
print(f'Mean Precipitation using numpy: {data_mean:.2f}')

# Max data
data_max = np.max(prec)
print(f'Max Precipitation: {data_max:.2f}')

# Min data
data_min = np.min(prec)
print(f'Min Precipitation: {data_min:.2f}')

# Variance data
data_var = np.var(prec)
print(f'Variance Precipitation: {data_var:.2f}')

# Standard deviation data
data_std = np.std(prec)
print(f'Standard Deviation Precipitation: {data_std:.2f}')

# sum data
data_sum = np.sum(prec)
print(f'Sum Precipitation: {data_sum:.2f}')

# Skewness data
data_skew = sps.skew(prec, bias=True)
print(f'Skewness Precipitation: {data_skew:.2f}')

# Kurtosis data
data_kurtosis = sps.kurtosis(prec, bias=True)
print(f'Kurtosis Precipitation: {data_kurtosis:.2f}')

# Percentile 90
percentile_90 = np.percentile(prec, 90)
print(f'Precentile 90: {percentile_90:.2f}')


## Menghitung jumlah data dengan sebuah kondisi
menggunakan `sum` dapat menghitung total jumlah data yang memenuhi syarat.

In [ ]:
# Menghitung jumlah hujan diatas nilai rata-rata
count_mean_rain = np.sum((prec >= data_mean))
print(f'Total precipitation greater than mean: {count_mean_rain}')

# Menghitung jumlah hujan di atas nilai percentile 90
count_percentile_rain = np.sum((prec >= percentile_90))
print(f'Total percipitation greater than Percentile 90: {count_percentile_rain}')

# Menghitung jumlah curah hujan diantara nilai rata-rata dan percentil 90
count_rain_conditionally = np.sum((prec >= data_mean) & (prec <=percentile_90))
print(f'Total percipitation conditionally: {count_rain_conditionally}')

## Plotting Timeseries menggunakan `Matplotlib`
Ploting time series dapat menggunakan `plot`. Selain itu, dapat menambah keterangan seperti `xlabel, ylabel, legend` dan `title`. `axhline` digunakan untuk menambahkan garis secara horizontal pada grafik. Selain itu, untuk membuat garis bentuknya berbeda-beda dapat menggantikan `linestyle` pada baris `axhline`.

In [ ]:
# Plot time series
plt.figure(figsize=(12, 6))
plt.plot(time, prec, label='Precipitation')
plt.title('Time Series Precipitation', fontsize=20)
plt.xlabel('time')
plt.ylabel('mm')
plt.axhline(data_mean, color='red', linestyle='--', label='mean')
plt.axhline(data_std, color='orange', linestyle=':', label='standard deviation')
plt.axhline(percentile_90, color='green', linestyle='-.', label='percentile 90')
plt.legend(loc='best')
plt.show()

## <tt>Latihan 1
### Buatkan timeseries Curah Hujan bulan Juni dan tambahkan garis percentile 98, mean, dan standard deviation

Langkah 1: select data di bulan Juni dengan menggunakan `dt.month` dan definisikan dengan nama baru.

Langkah 2: buatkan nilai persentil 98 dengan menggunakan `np.percentile`, untuk nilai mean dengan menggunakan `np.mean`, dan untuk nilai standard deviation dengan menggunakan `np.std`.

Langkah 3: Plot data dengan menggunakan `plt.plot` dan buat garis secara horizontal dengan menggunakan `plt.axhline`.


In [ ]:
"""Your code is here"""

## Plotting Histogram menggunakan `Matplotlib`
Ploting data menjadi sebuah distribusi dapat menggunakan `hist`. Dan untuk menambah garis keterangan secara vertikal dapat menggunakan `axvline`. Selain itu, bentuk dari `hist` dapat diubah sesuai keinginan kita dengan mengganti `histtype`.

In [ ]:
# ploting histogram 
plt.figure(figsize=(12, 6))
plt.hist(prec, 30, label='precipitation')
plt.title('Precipitation Histogram')
plt.xlabel('mm')
plt.ylabel('frequency')
plt.legend(loc='best')
plt.show()

# Ploting histogram with line
plt.figure(figsize=(12, 6))
plt.hist(prec, 30, label='precipitation', histtype='step')
plt.title('Precipitation Histogram')
plt.xlabel('mm')
plt.ylabel('frequency')
plt.legend(loc='best')
plt.axvline(data_mean, color='red', linestyle='--', label='mean')
plt.axvline(data_std, color='orange', linestyle=':', label='standard deviation')
plt.axvline(percentile_90, color='green', linestyle='-.', label='percentile 90')
plt.show()


## Plotting Histogram menggunakan `Seaborns`
Ploting histogram dengan menggunakan `seaborns` tidak berbeda jauh dibandingkan `matplotlib`, ini merupakan cara lain untuk menampilkan histogram. Ploting dapat menggunakan `histplot`. Selain itu, jumlah `bins` juga dapat diganti sesuai keinginan.

In [ ]:
# ploting histogram
plt.figure(figsize=(12, 6))
sns.histplot(prec, bins=30, label='precipitation', kde=True)
plt.title('Precipitation Histogram')
plt.xlabel('mm')
plt.ylabel('frequency')
plt.legend(loc='best')
plt.show()

## <tt>Latihan 2
### Buatkan distribusi curah hujan diatas 20 mm/day

Langkah 1: definisikan data hujan di atas 20 mm / day dengan nama baru.

Langkah 2: plot data menggunakan `plt.hist`.

In [ ]:
"""Your code is here"""

## Plotting Boxplot menggunakan `seaborns`
Ploting boxplot menggunakan `seaborns` dapat menggunakan `boxplot`. Ploting ini biasanya digunakan untuk melihat sebaran dari data tersebut.

In [ ]:
data_24['MONTH'] = data['DATE'].dt.month

# Plotting the boxplot for each month
plt.figure(figsize=(12, 6))
sns.boxplot(x='MONTH', y='RRRR', data=data_24, color='red')
plt.title('Monthly Rainfall Distribution')
plt.xlabel('Month')
plt.ylabel('Rainfall (mm)')
plt.grid(True)
plt.show()


## Menghitung korelasi
Terdapat 2 metode korelasi dan 2 libraries yang dapat digunakan, yaitu dengan menggunakan `pandas` dan `scipy`. Meskipun libraries yang digunakan berbeda, hasil yang didapatkan akan sama.

In [ ]:
# Correlation menggunakan pandas
cor_pearson = prec.corr(tmin, method='pearson')
print(f'Pearson correlation: {cor_pearson:.2f}')

cor_kendall = prec.corr(tmin, method='kendall')
print(f'Kendall correlation: {cor_kendall:.2f}')


# Correlation menggunakan scipy
pearson_coef, pearson_p = sps.pearsonr(prec, tmin)
print(f'Pearson correlation: {pearson_coef:.2f}')

kendall_coef, kendall_p = sps.kendalltau(prec, tmin)
print(f'Kendall correlation: {kendall_coef:.2f}')


## Menghitung regresi linear
Menghitung regresi linear dapat menggunakan `linregress`. Regresi linear biasanya mem-ploting data secara `scatter` (untuk melihat sebaran data) dan garis regresi tersebut.

In [ ]:
# Regresi linear
slope, intercept, r_value, p_value, std_err = sps.linregress(prec, tmin)

# Menghitung nilai slope yang baru
new_slope = prec * slope + intercept

print(f'slope value: {slope:.2f}')
print(f'intercept value: {intercept:.2f}')
print(f'new slope value: {np.mean(new_slope):.2f}')
print(f'r value: {r_value:.2f}')

# Ploting correlation, scatter
plt.figure(figsize=(12, 6))
plt.scatter(prec, tmin, label='data points')
plt.plot(prec, new_slope, color='red', label='regression')
plt.title('Precipitation-Min Temperature Correlation')
plt.xlabel('precipitation (mm)')
plt.ylabel('Min Temperature (C)')

# Display R-value on the plot
plt.plot([], [], ' ', label=f'R-value: {r_value:.2f}')
# plt.text(1, 1, f'R-value: {r_value:.2f}', fontsize=12, color='blue')

plt.legend(loc='upper right')
plt.show()

## <tt>Latihan 3
### Plot korelasi antara RRRR diatas 10 mm/day dan Tmin

Langkah 1: definisikan hujan diatas 10 mm dan buat dengan nama baru.

Langkah 2: definisikan data curah hujan dan suhu minimum masing-masing dengan nama baru

Langkah 3: menggunakan `sps.linregress` untuk mendapatkan linear regression agar mendapatkan nilai `slope, intercept,` dan `r`.

Langkah 4: Hitunglah nilai slope yang baru

Langkah 5: plot hasil dengan menggunakan `plt.scatter` untuk mengetahui sebarannya.

Langkah 6: plot garis regresinya dengan menggunakan `plt.plot`.



In [ ]:
"""Your code is here"""

## Menampilkan polinomial
Menghitung nilai polinomial dapat menggunakan `polyfit` dan `poly1d`. 

In [ ]:
polin1 = np.poly1d(np.polyfit(prec, tmin, 1))
polin2 = np.poly1d(np.polyfit(prec, tmin, 2))

# Ploting correlation, polinom
plt.figure(figsize=(12, 6))
plt.scatter(prec, tmin, label='data points')
plt.plot(prec, polin1(prec), color='red', label='regression 1 line')
plt.plot(prec, polin2(prec), color='green', label='regression 2 line')
plt.title('Precipitation-Min Temperature Correlation')
plt.xlabel('precipitation (mm)')
plt.ylabel('Min Temperature (C)')

plt.legend(loc='upper right')
plt.show()

## <tt>Latihan tambahan
### Plotting regresi 2 line antara curah hujan diatas 10 mm/day dan Tmin

Langkah 1: definisikan nilai polinomial dengan data curah hujan dan suhu minimum yang sudah diseleksi

Langkah 2: plot hasil dengan menggunakan `plt.scatter` untuk mengetahui sebarannya.

Langkah 3: plot garis regresinya dengan menggunakan `plt.plot`.

In [ ]:
"""Your code is here"""